In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from pandas.api.types import is_numeric_dtype
from pykalman import KalmanFilter
import re

In [2]:
df = pd.read_csv("final_train_data.csv.gz")

In [3]:
df.columns

Index(['timestamp', 'Latitude (°)', 'Longitude (°)', 'Height (m)',
       'Distance (cm)', 'Illuminance (lx)', 'amplitude', 'frequency',
       'Horizontal Accuracy (m)', 'Vertical Accuracy (m)',
       'Magnetic field x (µT)', 'Magnetic field y (µT)',
       'Magnetic field z (µT)', 'Acceleration x (m/s^2)',
       'Acceleration y (m/s^2)', 'Acceleration z (m/s^2)',
       'Gyroscope x (rad/s)', 'Gyroscope y (rad/s)', 'Gyroscope z (rad/s)',
       'Pressure (hPa)', 'Linear Acceleration x (m/s^2)',
       'Linear Acceleration y (m/s^2)', 'Linear Acceleration z (m/s^2)',
       'Common time (s)', 'Activity', 'Mood', 'Arousal ', 'Social engagement ',
       'Noise Level', 'Concentration Level'],
      dtype='object')

#### Support functions

In [122]:
def get_optimal_interval(dataframes: list[pd.DataFrame], time_col: str) -> float:
    """
    Find optimal time interval among datasets to avoid loosing data
    """

    min_intervals = []

    for df in dataframes:

        # calculate time between samples
        time_diffs = df[time_col].diff().dropna()

        # extract the most common interval size on the df
        min_intervals.append(time_diffs.mode()[0])

    # return optimal interval based on datasets
    return min(min_intervals)


def join_sdata(path: str, file_exception=None) -> pd.DataFrame:
    """
    Join sensor datasets into a cohessive dataset and save as CSV file.
    Adapt the new dataframe to work with optimal time interval on the 
    basis of the given datasets.
    """
    
    # generate path where datasets are located
    data_folder = Path(path)
    dataframes = []

    print("Files to be merged: \n")
    # iterate over files
    for file in data_folder.iterdir():
        if file.is_file() and file != file_exception:
            df = pd.read_csv(file)
            dataframes.append(df)
            print(f"- {file}: {len(df)} entries")
            

    # get time var from 
    time_var = next((col for col in dataframes[0].columns if "time" in col.lower()), None)

    # raise error if no time variabel found
    if time_var is None:
        raise KeyError("No time variable found. Data must be time frequency data.")

    # extrac max and min times from datasets
    min_time = min(df[time_var].min() for df in dataframes)
    max_time = max(df[time_var].max() for df in dataframes)
    print(f"\nMin time across datasets: {min_time}\nMax time across datasets: {max_time}")

    # create common time grid
    period_of_highest_freq = get_optimal_interval(dataframes, time_var) # 100 Hz by visual inspection; function is more precise
    common_time = pd.Index(np.arange(min_time, max_time, period_of_highest_freq))

    print(f"Highes fre. period: {period_of_highest_freq}\nLength of common_time: {len(common_time)}")

    # time adjusted dataframes
    adjusted_dfs = []

    # adjust each df to common time 
    for idx, df in enumerate(dataframes):
        
        # indexing df based on time
        time_indexed_df = df.set_index(time_var)

        # map old indices to common time indices
        mapped_indices = common_time.get_indexer(time_indexed_df.index, method="nearest")

        # re-index and impute values according to mapped indices
        time_resampled = time_indexed_df.reindex(common_time)
        time_resampled.iloc[mapped_indices] = time_indexed_df.values 

        # remove senor time variable 
        if time_var in time_resampled.columns:
            time_resampled.drop(time_var, axis=1)

        adjusted_dfs.append(time_resampled)

    # combine all files together 
    final_df = pd.concat(adjusted_dfs, axis=1)

    # add common time variable and index name
    final_df["Common time (s)"] = common_time
    final_df.index.name = "timestamp"

    return final_df

def add_labels(df: pd.DataFrame, labels: list[str], ctime: list[float], colname: str) -> pd.DataFrame:
    """
    Add activity labels to temporal dataframe
    """
    # create new colum for 
    df[colname] = None

    # get nearest time indices to the commulative measured 
    close_indices = df.index[df.index.get_indexer(ctime, method="nearest")]

    # fill in label for the cumulative time 
    for idx, (label, end_time) in enumerate(zip(labels, close_indices)):
        # define starting time for activeity 
        if idx == 0:
            start = df.index.min()
        else:
            start = ctime[idx - 1]
        
        # define ending time for activity
        if idx == len(ctime) - 1:
            end = df.index.max()
        else:
            end = end_time
        
        # fill in labels
        df.loc[start:end, colname] = label

    return df

def strtime_to_sec(str_time: str) -> float:
    """
    Convert 'min:sec,msec' to seconds (float)
    """

    min_, sec, msec = map(int, re.split(r"[:,]", str_time))

    total_time = (min_ * 60)  + sec + (msec / 1000)

    return total_time

def safe_interpol(df: pd.DataFrame, lb: float=0.25, ub: float=0.95, all: bool= False, edge_case: str="drop") -> pd.DataFrame:
    """
    Interpolate values in dataframe within a percentage range of missing values
    compared to the total amount of entries
    """

    for col in df.columns:
        
        # aply interpolation only if the variable is numeric 
        if pd.api.types.is_numeric_dtype(df[col]):
            
            # count nans per column
            nan_frac = df[col].isna().sum() / len(df)

            # interpolate columnns which have missing values within the range
            if (lb <= nan_frac <= ub) or all:
                df[col] = df[col].interpolate(method="linear")

                # if missing values -> edge cases (values at start or end: between NaN's)
                if df[col].isna().any():

                    # fill in with neares forward or backwards value
                    if edge_case == "fill":
                        df[col] = df[col].bfill().ffill()
                    # fill in missing values with zeros
                    elif edge_case == "zero":
                        df[col] = df[col].fillna(0)
                    # do nothing 
                    elif edge_case == "drop":
                        pass
                
    return df

# support fucntions
def one_d_kalman_filter(df):
    """basic 1d kalman filterfunction based on the function provided by the professor (sigh)"""
    for i,column in enumerate(df.columns):
        print(f"{i} column out of {len(df.columns)}")
        if not is_numeric_dtype(df[column]):
            continue
        else:
            kf =  KalmanFilter(transition_matrices=[[1]], observation_matrices=[[1]]) #transition and observation matrices for computations
            masked_values = np.ma.masked_invalid(df[column].values.astype(np.float32))

            kf_params = kf.em(masked_values,n_iter=3) #optimizes Q and R (the noise estimates) and so on of the kalman filter to improve outlier detection and imputation
            imputed_data, covariances = kf_params.smooth(masked_values) #applies the filter
            df[column] = imputed_data
    return df

def multivariate_kalman_filter(df):
    """updates function from professor to multivariate kalman filter"""
    columns_to_impute = [column for column in df.columns if  is_numeric_dtype(df[column])]
    kf =KalmanFilter(
        transition_matrices = np.eye(len(columns_to_impute)), #initialize transition matrix to just next state
        observation_matrices = np.eye(len(columns_to_impute)), #initialize observation matrix to just next state
        transition_covariance = np.eye(len(columns_to_impute))*0.5, #initialize Q to moderate uncertainty about transition model
        observation_covariance = np.eye(len(columns_to_impute)) *0.5)#initialize R to moderate uncertainty about measurements 
    
    masked_df = np.ma.masked_invalid(df[columns_to_impute].values.astype(np.float32))
    kf_params = kf.em(masked_df,n_iter=3) #optimizes Q and R (the noise estimates) and so on of the kalman filter to improve outlier detection and imputation
    imputed_data, covariances = kf_params.smooth(masked_df) #applies the filter
    print("imputation of multivariate done")
    for i, column in enumerate(columns_to_impute):
        df[column] = imputed_data[:,i]
    return df

def mixed_kalman_filter(df):
    """function that applies multivariate approach for highly correlated features and univariate for others"""
    print("now correlated")
    correlated_columns = df[["Latitude (°)", "Longitude (°)", "Height (m)"]]
    correlated_df = multivariate_kalman_filter(correlated_columns)

    print("now uncorrelated")
    uncorrelated_columns = df.drop(columns=correlated_columns.columns)
    uncorrelated_df = one_d_kalman_filter(uncorrelated_columns)

    df_concat = pd.concat([correlated_df, uncorrelated_df], axis=1)
    return df_concat


#### Generate joined dataset

In [123]:
# join data frames, add labels and save
joined_data = join_sdata("train_data", file_exception="train_data\\Proximity.csv")
joined_data.head()

Files to be merged: 

- train_data/Proximity.csv: 7 entries
- train_data/Light.csv: 300 entries
- train_data/audio.csv: 96541 entries
- train_data/Location.csv: 53 entries
- train_data/Magnetometer.csv: 103422 entries
- train_data/Accelerometer.csv: 416321 entries
- train_data/Gyroscope.csv: 416316 entries
- train_data/Pressure.csv: 41229 entries
- train_data/Linear Acceleration.csv: 50893 entries

Min time across datasets: 0.0
Max time across datasets: 1035.037413
Highes fre. period: 0.0024862999999868407
Length of common_time: 416297


,Distance (cm),Illuminance (lx),amplitude,frequency,Latitude (°),Longitude (°),Height (m),Velocity (m/s),Direction (°),Horizontal Accuracy (m),...,Acceleration y (m/s^2),Acceleration z (m/s^2),Gyroscope x (rad/s),Gyroscope y (rad/s),Gyroscope z (rad/s),Pressure (hPa),Linear Acceleration x (m/s^2),Linear Acceleration y (m/s^2),Linear Acceleration z (m/s^2),Common time (s)
timestamp,,,,,,,,,,,,,,,,,,,,,
0.000000,NaN,126.14875,0.003599,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
0.002486,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.002486
0.004973,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.004973
0.007459,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.510732,7.223082,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.007459
0.009945,NaN,NaN,0.007781,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,6.535572,7.266178,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.009945


In [124]:
joined_data.isna().sum()

Distance (cm)                    416290
Illuminance (lx)                 415997
amplitude                        319756
frequency                        319756
Latitude (°)                     416244
Longitude (°)                    416244
Height (m)                       416246
Velocity (m/s)                   416297
Direction (°)                    416297
Horizontal Accuracy (m)          416244
Vertical Accuracy (m)            416244
Magnetic field x (µT)            312875
Magnetic field y (µT)            312875
Magnetic field z (µT)            312875
Acceleration x (m/s^2)              333
Acceleration y (m/s^2)              333
Acceleration z (m/s^2)              333
Gyroscope x (rad/s)                 338
Gyroscope y (rad/s)                 338
Gyroscope z (rad/s)                 338
Pressure (hPa)                   375068
Linear Acceleration x (m/s^2)    365404
Linear Acceleration y (m/s^2)    365404
Linear Acceleration z (m/s^2)    365404
Common time (s)                       0


Missing values as expected

#### Add activities

In [125]:
# activities in sequence of occurance 
activ_seq = ["rest", "walk", "phone", "stairs", "rest", 
                "phone", "socialize", "walk", "study", "walk",
                "stairs", "walk", "phone", "study", "socialize"]
# relative activity times
rtimes = ["01:15,13", "00:50,92", "01:01,08", "1:17,82", "01:15,43",
                    "00:50,37", "01:15,53", "00:42,41", "01:16,86", "00:55,36",
                    "01,29,60", "01:02,60", "00:40,60", "01:45,82", "01:26,80"]
rtimes_sec = [strtime_to_sec(t) for t in rtimes]

# actual time (cummulative time)
linear_time = [rtimes_sec[0]]
for idx, val in enumerate(rtimes_sec[1:]):
    linear_time.append(val + linear_time[idx])


In [126]:
labeld_data = add_labels(joined_data, activ_seq, linear_time, "Activity")
labeld_data["Activity"].value_counts()

Activity
walk         84153
study        72867
socialize    71326
stairs       66823
phone        60775
rest         60353
Name: count, dtype: int64

Activites added correctly!

In [127]:
x = pd.Index(np.arange(0, 10, 0.5))
y = [1.66, 7.8, 0.5, 8.88, 5.28]
mapped_y = x.get_indexer(y, method="nearest")
print(f"x ={x.values}")
print(f"y = {y}")
print(f"Mapped_y {mapped_y}")
print(f"Index mapping: {x[mapped_y].values}")

x =[0.  0.5 1.  1.5 2.  2.5 3.  3.5 4.  4.5 5.  5.5 6.  6.5 7.  7.5 8.  8.5
 9.  9.5]
y = [1.66, 7.8, 0.5, 8.88, 5.28]
Mapped_y [ 3 16  1 18 11]
Index mapping: [1.5 8.  0.5 9.  5.5]


#### Add survey values

In [128]:
surv_data = pd.read_csv("responses_survey.csv")
surv_data.head()

,Timestamp,Mood,Arousal,Social engagement,Noise Level,Concentration Level
0,6/5/2025 16:26:37,3,2,1,3,1
1,6/5/2025 16:27:33,3,3,1,2,1
2,6/5/2025 16:28:34,3,2,1,2,1
3,6/5/2025 16:29:40,3,3,1,2,1
4,6/5/2025 16:30:35,3,2,1,1,1


In [129]:
# add relative time experiment 
surv_data["Linear time"] = None
start_time = None

for idx, time_val in enumerate(surv_data.iloc[:, 0]):
    # extract time from timestamp and convert to min:sec,msec
    time = time_val.split()[1][3:] + ",00"
    
    # get relative time 
    if start_time is None: start_time = strtime_to_sec(time)

    linear_time = strtime_to_sec(time) - start_time

    surv_data.loc[idx, "Linear time"] = linear_time

surv_data.head()

,Timestamp,Mood,Arousal,Social engagement,Noise Level,Concentration Level,Linear time
0,6/5/2025 16:26:37,3,2,1,3,1,0.0
1,6/5/2025 16:27:33,3,3,1,2,1,56.0
2,6/5/2025 16:28:34,3,2,1,2,1,117.0
3,6/5/2025 16:29:40,3,3,1,2,1,183.0
4,6/5/2025 16:30:35,3,2,1,1,1,238.0


In [130]:
# add labels from survey data
ctime = surv_data["Linear time"]

# loop over each variable to add labels 
for col in surv_data.columns[1:-1]:
    add_labels(joined_data, surv_data[col], ctime, col)

# check imputation of labels
for col in surv_data.columns[1:-1]:
    print(f"{col} NaN's: {joined_data[col].isna().sum()}")

Mood NaN's: 0
Arousal  NaN's: 0
Social engagement  NaN's: 0
Noise Level NaN's: 0
Concentration Level NaN's: 0


Correct assignment of value

In [131]:
# view of current dataset
joined_data.head()

,Distance (cm),Illuminance (lx),amplitude,frequency,Latitude (°),Longitude (°),Height (m),Velocity (m/s),Direction (°),Horizontal Accuracy (m),...,Linear Acceleration x (m/s^2),Linear Acceleration y (m/s^2),Linear Acceleration z (m/s^2),Common time (s),Activity,Mood,Arousal,Social engagement,Noise Level,Concentration Level
timestamp,,,,,,,,,,,,,,,,,,,,,
0.000000,NaN,126.14875,0.003599,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.000000,rest,3,3,1,2,1
0.002486,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.002486,rest,3,3,1,2,1
0.004973,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.004973,rest,3,3,1,2,1
0.007459,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.007459,rest,3,3,1,2,1
0.009945,NaN,NaN,0.007781,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.009945,rest,3,3,1,2,1


In [132]:
joined_data.isna().sum()

Distance (cm)                    416290
Illuminance (lx)                 415997
amplitude                        319756
frequency                        319756
Latitude (°)                     416244
Longitude (°)                    416244
Height (m)                       416246
Velocity (m/s)                   416297
Direction (°)                    416297
Horizontal Accuracy (m)          416244
Vertical Accuracy (m)            416244
Magnetic field x (µT)            312875
Magnetic field y (µT)            312875
Magnetic field z (µT)            312875
Acceleration x (m/s^2)              333
Acceleration y (m/s^2)              333
Acceleration z (m/s^2)              333
Gyroscope x (rad/s)                 338
Gyroscope y (rad/s)                 338
Gyroscope z (rad/s)                 338
Pressure (hPa)                   375068
Linear Acceleration x (m/s^2)    365404
Linear Acceleration y (m/s^2)    365404
Linear Acceleration z (m/s^2)    365404
Common time (s)                       0


In [133]:
# save dataset
joined_data.to_csv("final_train_data_unimputed.csv")

#### Impute missing values using interpolation and Kalman filter

In [134]:
# interpolate missing values
final_data = joined_data.copy()
final_data = safe_interpol(final_data, edge_case="fill")
final_data.head()

,Distance (cm),Illuminance (lx),amplitude,frequency,Latitude (°),Longitude (°),Height (m),Velocity (m/s),Direction (°),Horizontal Accuracy (m),...,Linear Acceleration x (m/s^2),Linear Acceleration y (m/s^2),Linear Acceleration z (m/s^2),Common time (s),Activity,Mood,Arousal,Social engagement,Noise Level,Concentration Level
timestamp,,,,,,,,,,,,,,,,,,,,,
0.000000,NaN,126.14875,0.003599,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.064004,0.030721,0.083742,0.000000,rest,3,3,1,2,1
0.002486,NaN,NaN,0.004645,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.064004,0.030721,0.083742,0.002486,rest,3,3,1,2,1
0.004973,NaN,NaN,0.005690,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.064004,0.030721,0.083742,0.004973,rest,3,3,1,2,1
0.007459,NaN,NaN,0.006736,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.064004,0.030721,0.083742,0.007459,rest,3,3,1,2,1
0.009945,NaN,NaN,0.007781,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.064004,0.030721,0.083742,0.009945,rest,3,3,1,2,1


In [135]:
final_data.isna().sum()

Distance (cm)                    416290
Illuminance (lx)                 415997
amplitude                             0
frequency                             0
Latitude (°)                     416244
Longitude (°)                    416244
Height (m)                       416246
Velocity (m/s)                   416297
Direction (°)                    416297
Horizontal Accuracy (m)          416244
Vertical Accuracy (m)            416244
Magnetic field x (µT)                 0
Magnetic field y (µT)                 0
Magnetic field z (µT)                 0
Acceleration x (m/s^2)              333
Acceleration y (m/s^2)              333
Acceleration z (m/s^2)              333
Gyroscope x (rad/s)                 338
Gyroscope y (rad/s)                 338
Gyroscope z (rad/s)                 338
Pressure (hPa)                        0
Linear Acceleration x (m/s^2)         0
Linear Acceleration y (m/s^2)         0
Linear Acceleration z (m/s^2)         0
Common time (s)                       0


In [136]:
remove_cols = []
n_data = len(final_data)

# check for missing values in final data set
for col in final_data.columns:
    nans = final_data[col].isna().sum()
    if nans:
        print(f"{col} has {nans}")
        if nans == n_data:
            remove_cols.append(col)

print("\nThe following variables have no values:")
for var in remove_cols:
    print(var)

Distance (cm) has 416290
Illuminance (lx) has 415997
Latitude (°) has 416244
Longitude (°) has 416244
Height (m) has 416246
Velocity (m/s) has 416297
Direction (°) has 416297
Horizontal Accuracy (m) has 416244
Vertical Accuracy (m) has 416244
Acceleration x (m/s^2) has 333
Acceleration y (m/s^2) has 333
Acceleration z (m/s^2) has 333
Gyroscope x (rad/s) has 338
Gyroscope y (rad/s) has 338
Gyroscope z (rad/s) has 338

The following variables have no values:
Velocity (m/s)
Direction (°)


In [137]:
# drop empty variables
final_data.drop(columns=remove_cols, inplace=True)

Now that we interpolated everything that is possible, we will use Kalman Filter

In [138]:
final_data = mixed_kalman_filter(final_data)

now correlated


KeyboardInterrupt: 

In [ ]:
final_data.head()

,Latitude (°),Longitude (°),Height (m),Distance (cm),Illuminance (lx),amplitude,frequency,Horizontal Accuracy (m),Vertical Accuracy (m),Magnetic field x (µT),...,Linear Acceleration x (m/s^2),Linear Acceleration y (m/s^2),Linear Acceleration z (m/s^2),Common time (s),Activity,Mood,Arousal,Social engagement,Noise Level,Concentration Level
timestamp,,,,,,,,,,,,,,,,,,,,,
0.000000,1.257433,0.116927,0.069011,0.444559,67.119613,0.003641,3.964895e-19,0.169948,0.024887,18.961148,...,0.061589,0.029563,0.080543,0.001231,rest,3,3,1,2,1
0.002486,1.414592,0.131541,0.077637,0.555727,67.184122,0.004651,3.616358e-20,0.212429,0.031108,19.588866,...,0.063213,0.030341,0.082709,0.002898,rest,3,3,1,2,1
0.004973,1.571751,0.146155,0.086262,0.666895,67.248631,0.005686,3.298460e-21,0.254909,0.037328,19.766805,...,0.063745,0.030597,0.083409,0.005110,rest,3,3,1,2,1
0.007459,1.728910,0.160770,0.094887,0.778062,67.313140,0.006703,3.008506e-22,0.297390,0.043549,19.817245,...,0.063919,0.030680,0.083634,0.007505,rest,3,3,1,2,1
0.009945,1.886069,0.175384,0.103513,0.889230,67.377649,0.007591,2.744041e-23,0.339871,0.049770,19.831543,...,0.063976,0.030708,0.083707,0.009961,rest,3,3,1,2,1


In [ ]:
final_data.to_csv("final_train_data.csv")